### 1. Импортируеми библиотеки

In [2]:
import pandas as pd
import requests
import csv
from  bs4 import BeautifulSoup as bs
from urllib.request import urljoin

### 2. Запускаем сбор урлов категорий

In [43]:
# Ф1
# ====================================Задаем параметры сессии
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/'
          }
domen = 'https://winestyle.ru/'
url = 'https://winestyle.ru/'
session = requests.Session()
r = session.get(url, headers=headers)
soup = bs(r.text, 'lxml')
# ====================================Завершили параметры сессии
links = []
urls=[]
ul = soup.find('ul', class_='menu menu-main clearfix')
if ul:
    for li in ul.find_all('li'):
        a = li.find('a', href=True)
        if a:
            urls.append(a['href'])
for u in urls:
    link  = urljoin(domen,u)
    links.append(link)
display(links)

['https://winestyle.ru/promo/discount/',
 'https://winestyle.ru/wine/',
 'https://winestyle.ru/champagnes-and-sparkling/',
 'https://winestyle.ru/whisky/',
 'https://winestyle.ru/cognac/',
 'https://winestyle.ru/beer/',
 'https://winestyle.ru/liqueur/',
 'https://winestyle.ru/vodka/',
 'https://winestyle.ru/water/',
 'https://winestyle.ru/wine-set/',
 'https://winestyle.ru/glass/',
 'https://winestyle.ru/accessory/',
 'https://winestyle.ru/chocolate/',
 'https://winestyle.ru/rum/']

### 3. Выбор категорий для парсинга

In [42]:
# Ф2.1 доп.
links2=[]
links2.append(links[4])
links2.append(links[3])
links2

['https://winestyle.ru/cognac/', 'https://winestyle.ru/whisky/']

### 4. Запускаем сбор пагинации

In [ ]:
# Ф2
pagination = []
for i in links2: # заменить на links2 при тестировании
    u = str(i)+str('all/?page=')
    first_page = str(i)+str('all')
    pagination.append(first_page)
    p = 2
    pagi_url='https://winestyle.ru/'
    while status(pagi_url) == 200:
        pagi_url = str(u)+str(p)
        display(pagi_url)
        pagination.append(pagi_url)
        p+=1
    else:
        pagination.remove(pagi_url)
        save(pagination)
        print(pagination[-1])
        pagination = []
        continue

In [ ]:
# Ф2.2 доп. (Проверка наличия страницы пагинации)
def status (url):
    headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/'
          }
    session = requests.Session()
    r = session.get(url, headers=headers)
    response = r.status_code
    return response

# Ф2.3 доп. (Сохранение промежуточных результатов)
def save (result):
    #import csv
    filename = str(pagination[0][21:]).replace('/','.')+str('.csv')
    with open(filename, 'w') as outfile:
        spamwriter = csv.writer(outfile, lineterminator='\n')
        spamwriter.writerow(['Page_Link']) 
        for i in result:
            spamwriter.writerow([i])  

### Загружаем пагинацию выбранной категории

In [51]:
# Ф3
results=[]
with open('../cognac.all.csv', 'r') as data:
    data.readline().rstrip()
    for line in data:
        results.append(line[:-1])  

### Запускаем сбор урлов целевых страниц

In [ ]:
# Ф4
# ====================================Задаем параметры сессии====================
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/'
          }
domen = 'https://winestyle.ru/'
#url = 'https://winestyle.ru/'            /объявляется внутри цикла for
session = requests.Session()
#r = session.get(url, headers=headers)   /объявляется внутри цикла for
#soup = bs(r.text, 'lxml')               /объявляется внутри цикла for
# ====================================Завершили параметры сессии===================
products_links=[]
for url in results: #для полного прогона
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    if soup != None:
        t_links = soup.find_all('p', class_="title")
        for i in t_links:
            pl = i.find('a', href=True)
            d = urljoin(domen,pl['href'])
            products_links.append(d)
            print(str(url[-3:])+str(' ')+str(d))
save_products(products_links) #вызываем функцию сохранения результатов

In [55]:
# Ф4. Доп. (сохраняем ссылки на целевые страницы в csv файл)
def save_products (result):
    with open('product_links.csv', 'w') as outfile:
        spamwriter = csv.writer(outfile, lineterminator='\n')
        spamwriter.writerow(['Product_Link']) 
        for i in result:
            spamwriter.writerow([i])

### Загружаем урлы целевых страниц 

In [57]:
# Ф5
results=[]
with open('../product_links.csv', 'r') as data:
    data.readline().rstrip()
    for line in data:
        results.append(line[:-1]) 

### Запускаем парсинг целевых страниц

In [ ]:
# Ф6
# ====================================Задаем параметры сессии====================
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/'
          }
#domen = 'https://winestyle.ru/'          /здесь не нужен
#url = 'https://winestyle.ru/'            /объявляется внутри цикла for
session = requests.Session()
#r = session.get(url, headers=headers)   /объявляется внутри цикла for
#soup = bs(r.text, 'lxml')               /объявляется внутри цикла for
# ====================================Завершили параметры сессии===================
products=[] # список, куда будем вставлять словари product
for url in results:
    product={}  # словарь для занесения параметров
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')  
    try:
        if soup:
            articul = soup.find('span', class_='bg-text')
            if articul:
                product['Артикул'] = str(articul.text).lstrip('Артикул:')
            title = soup.find('h1').text
            if title:
                product['Название'] = title.strip()            
            price = soup.find('div', class_='price').text
            if price:
                product['Цена'] = price.strip()
            params = soup.find('ul', class_='list-description list-description-lined').find_all('li')
            fr = soup.find('ul', class_='list-description list-description-lined').text
            dm = fr.strip().split('\n\n')
            count=1
            l1=[]
            l2=[]
            dict1={}
            for i in dm:
                i.strip().replace(' ','')
                if count%2 == 0:
                    l1.append(i)
                else:
                    l2.append(i.strip())
                count +=1
            test_dict={}
            test_dict = dict(zip(l2, l1))
            product.update(test_dict)
            product['Страница']=url                       
            pic = soup.find('a', class_='img-container fancybox', href=True)['href']
            if pic:
                product['Изображение'] = pic.strip()     
            score = soup.find('div', class_='info-block rating-text rating-text-big').find('span', class_='text').text
            if score:
                product['Рейтинг'] = score.strip()
    except  AttributeError :
         pass
    products.append(product)
save_product_info(products)

In [61]:
# Ф6.1 Доп.  (собираем названия полей с словарях с продуктами для формирования заголовка в CSV)
def get_field_name(products):
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    return field_names

In [73]:
# Ф6.2 Доп. (Сохраняем финальные результаты)
def save_product_info(products):
    with open('../products.csv', 'w') as outfile:
        spamwriter = csv.DictWriter(outfile, fieldnames=get_field_name(products), lineterminator='\n')    
        spamwriter.writeheader() #записывет заголовки столбцов равные ключам словаря
        spamwriter.writerows(products)